# Agenda

1. Iterator protocol
    - Adding iterations to classes
    - Generator functions
    - Generator comprehensions
2. Decorators
3. Threading and multiprocessing

# `__str__` vs. `__repr__`

Both of these methods should return strings. The question is, when is each one run?

Normally `__str__` is run:
- Whenever we run `str` on something
- If we use `print` on something (because it uses `str` behind the scenes)
- In other words: When we want to turn our object into a string, to display it to end users

Normally, `__repr__` is run:
- If we're in a debugger
- If we're in Jupyter (and not using `print`, but just asking for the value of a var